In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data2= pd.read_csv('/kaggle/input/102-ml-hw2-data/train/train.csv')

In [ ]:
print(data2.info())

In [ ]:
data=data2.copy()

In [ ]:
!pip3 install nltk
!pip3 install symspellpy
!pip3 install pandas

In [ ]:
import re
def pbx_into_sp(org_str) : 
    org_str = re.sub('\W+', ' ', org_str).replace("_", '').strip()#將句子中的特殊符號以空格取代
    return org_str
# 建立新欄位'clear_comment'存放清理好的評論
data['clear_text'] = data['text'].apply(lambda x: pbx_into_sp(x))

In [ ]:
print(data.head())

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
# 進行停用詞過濾
# 下載NLTK的停用詞列表
nltk.download('punkt')  #用于下载NLTK的标点符号工具
nltk.download('stopwords') #用于下载英语停用词列表
nltk_stopwords = nltk.corpus.stopwords.words('english')

def rmsw_function(text, stopword_list):
    return ' '.join([word for word in word_tokenize(text) if word not in stopword_list])
    #这个过程的目标是从文本中去除那些在文本分类或自然语言处理任务中通常没有信息量的常见词汇，
    #例如 "the"、"and"、"in" 等。这可以帮助模型更好地关注文本中的重要内容。

# 使用前面初步清理過的clear_comment來過濾停用詞
# 建立新欄位'clear_comment_sw'存放過濾停用詞的評論
data['clear_text_sw'] = data['clear_text'].apply(lambda x: rmsw_function(x, nltk_stopwords))

In [ ]:
print(data.head())

In [ ]:
#文本長度，即文本總字數。
def get_txt_len(txt):
    txt_list = txt.split(' ')
    return len(txt_list)
    
data['txt_len'] = data['clear_text_sw'].apply(lambda x: get_txt_len(x))

In [ ]:
#文本中的词汇频率和词汇量（不重复的词汇数量）
def get_frequency(txt):
    words = nltk.word_tokenize(txt)
    word_frequency = nltk.FreqDist(words)
    vocabulary_size = len(word_frequency)
    return vocabulary_size
    
data['word_frequency'] = data['clear_text_sw'].apply(lambda x: get_frequency(x))

In [ ]:
#文本中的全英大寫字詞出現次數。
def get_cap_n(txt):
    big=0
    txt_list = txt.split()
    for t in txt_list:
        if t.isupper():
            big+=1
    return big
    
data['cap_n'] = data['clear_text_sw'].apply(lambda x: get_cap_n(x))

In [ ]:
#文本中的句子總數。
def get_sen_n(org_txt):
    # 會有人用多種符號來區隔句子，因此使用多種符號分割評論
    sen_list = re.split(r';|/|\!|\n|\?|\.', org_txt)
    # 過濾掉空字串
    sen_list = list(filter(None, sen_list))
    return len(sen_list)

# 特別注意要使用清理前的字串作為輸入欄位
data['sen_n'] = data['text'].apply(lambda x: get_sen_n(x))

In [ ]:
print(data.info())

In [ ]:
print(data.head())

情感分數的數值範圍介於-1至1之間，數值趨近於-1顯示該文本平均屬於負面情感，趨近於1則顯示該文本平均屬於正面情感，若趨近於0的數值則代表情感表現不明顯，為中性情感文本。

In [ ]:
from textblob import TextBlob
def get_pos_sentiment_n(org_txt):
    positive_sentiment_return=0
    sentiment = TextBlob(org_txt)
    for sentence in sentiment.sentences:
        positive_sentiment_return = positive_sentiment_return+sentiment.sentiment.polarity
    return positive_sentiment_return

def get_neg_sentiment_n(org_txt):
    negative_sentiment_return=0
    sentiment = TextBlob(org_txt)
    for sentence in sentiment.sentences:
        negative_sentiment_return = negative_sentiment_return+sentiment.sentiment.subjectivity
    return negative_sentiment_return

data['pos_sentiment_n'] = data['text'].apply(lambda x: get_pos_sentiment_n(x))
data['neg_sentiment_n'] = data['text'].apply(lambda x: get_neg_sentiment_n(x))

In [ ]:
print(data.head())

In [ ]:
from nltk.corpus import stopwords
import string
eng_stopwords=set(stopwords.words('english'))

## Number of characters in the text ##
data["num_chars"] = data["text"].apply(lambda x: len(str(x)))

## Number of stopwords in the text ##
data["num_stopwords"] = data["text"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))

## Number of punctuations 標點 in the text ##
data["num_punctuations"] =data['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )

## Number of title case words in the text ##
data["num_words_title"] = data["text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))

## Average length of the words in the text ##
data["mean_word_len"] = data["text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

In [ ]:
print(data.head())

In [ ]:
#data.to_csv('S_ver3.csv', index=False)

In [ ]:
'''
from textblob import TextBlob

text = '''The titular threat of The Blob has always struck me as the ultimate movie
monster: an insatiably hungry, amoeba-like mass able to penetrate
virtually any safeguard, capable of--as a doomed doctor chillingly
describes it--assimilating flesh on contact.
Snide comparisons to gelatin be damned, it is a concept with the most
devastating of potential consequences, not unlike the grey goo scenario
proposed by technological theorists fearful of
artificial intelligence run rampant.
'''

blob = TextBlob(text)
print(blob, sentence.sentiment.polarity, sentence.sentiment.subjectivity)
for sentence in blob.sentences:
    print(sentence, sentence.sentiment.polarity, sentence.sentiment.subjectivity)
    '''